In [63]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
import statsmodels.formula.api as smf

In [79]:
data = pd.read_csv("https://raw.githubusercontent.com/fivethirtyeight/data/master/masculinity-survey/masculinity-survey.csv")

one_hot = pd.read_csv("one-hot-responses.csv")
# Very masculine, Somewhat masculine, Not very masculine, Not at all masculine
masculinity_scale = [col for col in one_hot.columns if col.startswith('q0001')]
# Straight, Gay, Bisexual, Other
sexuality_scale = [col for col in one_hot.columns if col.startswith('q0026')]
# print(masculinity_scale)
# print(sexuality_scale)

predictors = [one_hot.columns[i] for i in range(7, len(one_hot.columns))]
X = one_hot[predictors].values.T
n_i, n_j = X.shape
stds = np.std(X, axis=1)
stds = np.repeat(np.reshape(stds, (-1,1)), n_j, axis=1)
S = X/stds
pcs = PCA(n_components=20).fit_transform(S.T)